In [ ]:
# leadfield_functions.py
# Description: Main script for leadfield generation.
# Author: Mohammad Orabe
#         ZiXuan Liu
#         Nikola Jajcay
#
# Contact:
#          orabe.mhd@gmail.com
#          a837707601@gmail.com
#          jajcay@ni.tu-berlin.de
#          martink@bccn-berlin.de

In [ ]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt

from leadfield_functions import *


In [ ]:
subject = "fsaverage"
atlas_nii_path = "AAL2/atlas/AAL2.nii"
atlas_xml_path = "AAL2/atlas/AAL2.xml"


In [ ]:
subjects_dir, trans, fs_dir, raw, atlas_nii, atlas_xml = load_data(atlas_nii_path, atlas_xml_path)


In [ ]:
# Manually build up BEM
# conductivity = (0.3,)  # for single layer
conductivity = (0.3, 0.006, 0.3)  # for three layers
bem = build_BEM(subject, subjects_dir, fs_dir, conductivity=conductivity)


In [ ]:
plot_bem_kwargs = dict(
    subject=subject,
    subjects_dir=subjects_dir,
    brain_surfaces="white",
    orientation="coronal",
    slices=[50, 100, 150, 200],
)
mne.viz.plot_bem(**plot_bem_kwargs)
plt.show()

In [ ]:
# Generate surface source space
## The method to achieve AAL atlas source space is to calculate the average dipole value of all dipoles in each annotation of the atlas, so at first place, the surface source space needed to be generated. The data "fsaverage" has a pre build-up surface source space, nevertheless, the source space can also be calculated.
src = os.path.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
mne.viz.plot_bem(src=src, **plot_bem_kwargs)
plt.show()

In [ ]:
# Clean channel names to be able to use a standard 1020 montage
clean_eeg_channels(raw)

In [ ]:
plot_EEG_montage(raw, src, trans)

In [ ]:
leadfield_downsampled, fwd, unique_labels = compute_leadfield(
    raw,
    trans,
    src,
    bem,
    subject,
    atlas_nii,
    atlas_xml,
    atlas="aal2_cortical",
    cortex_parts="full_cortex",
    path_to_save=os.getcwd(),
)


In [ ]:
check_atlas_missing_regions(atlas_xml_path, unique_labels)


In [ ]:
# verify_leadfield(subject, subjects_dir, fwd, location="center", extent = 10.0)
